[Home](../index.ipynb) / Rotary encoder
***
<span style="font-size:20pt;">Rotary encoder</span>

In [ ]:
#%serialconnect --port=COM3 --baud=115200 
%serialconnect

In [ ]:
# Robust Rotary encoder reading
# Copyright John Main - best-microcontroller-projects.com
# Adapted to Python - Alfred Fuchs
# https://www.best-microcontroller-projects.com/rotary-encoder.html
# Improved Table Decode

from machine import Pin

class RotaryEncoder :
    _rot_enc_table = [0,1,1,0,1,0,0,1,1,0,0,1,0,1,1,0]
    
    def __init__( self, _iPinCLK, _iPinDT, _iLeftBound = None, _iRightBound = None, _bCycle = False, _bNoisyEncoder = True ) :
        self._bNoisyEncoder = _bNoisyEncoder
        self._pinCLK        = Pin( _iPinCLK, Pin.IN, Pin.PULL_UP )
        self._pinDT         = Pin( _iPinDT,  Pin.IN, Pin.PULL_UP )
        self._prevNextCode  = 0
        self._store         = 0
        self._value         = 0
        self._iLeftBound    = _iLeftBound
        self._iRightBound   = _iRightBound
        self._bCycle        = _bCycle


    # A valid CW or CCW move returns 1, invalid returns 0.
    def read_rotary( self ) :
        self._prevNextCode <<= 2

        if self._pinDT .value() == 0 : self._prevNextCode |= 0x02
        if self._pinCLK.value() == 0 : self._prevNextCode |= 0x01

        self._prevNextCode &= 0x0f


        # If valid then store as 16 bit data.
        if RotaryEncoder._rot_enc_table[self._prevNextCode] != 0 :
            self._store = int( self._store << 4 )
            self._store |= self._prevNextCode
            self._store &= 0xffff
            
            if self._bNoisyEncoder == True :
                if (self._store & 0xff) == 0x2b: return -1
                if (self._store & 0xff) == 0x17: return 1
            else :
                if self._store == 0xd42b : return  1
                if self._store == 0xe817 : return -1

        return 0;

    def update( self ) :
        step = self.read_rotary()

        if step != 0:
            self._value += step
            
            if step == 1 :
                if self._iRightBound is not None and self._value > self._iRightBound :
                    self._value = self._iLeftBound if self._bCycle else self._iRightBound

            elif step == -1 :
                if self._iLeftBound is not None and self._value < self._iLeftBound :
                    self._value = self._iRightBound if self._bCycle else self._iLeftBound
            
            return True
        
        else :
            return False
        
    def value( self ) :
        return self._value
    
    def __str__(self) :
        strDirection = "<--" if self._prevNextCode == 0x0b else "-->" if self._prevNextCode == 0x07 else "???"

        return ( strDirection + " value = " + str( self._value ) + ", store = " + hex(self._store) + ", prevNextCode = " + hex( self._prevNextCode ) )

In [ ]:
# Sample
rotaryEncoder = RotaryEncoder( 2, 22 )

from display import Display
import time

display = Display()

try:
    while True:
        if rotaryEncoder.update() == True:
            display.fill_rect( 0, 0, 127, 10, 0 )
            display.text( f"{rotaryEncoder.value()}", 0, 0 )
            display.show()

except KeyboardInterrupt:
    pass

display.text( "Done.", 0, 50 )
display.show()
print( "\nDone." )

**Sources:**

* [best-microcontroller-projects.com](https://www.best-microcontroller-projects.com/rotary-encoder.html)
* [mikrocontroller.net](https://www.mikrocontroller.net/articles/Drehgeber)
* [randomnerdtutorials.com](https://randomnerdtutorials.com/micropython-interrupts-esp32-esp8266/)
* [micropython.org](https://docs.micropython.org/en/latest/reference/isr_rules.html)


|Prev/Next State|Valid code|Direction|
|------|---|---|
| 0000 | X | X |
| 0001 | Valid | CW |
| 0010 | Valid | CCW |
| 0011 | X | X |
| 0100 | Valid | CCW |
| 0101 | X | X |
| 0110 | X | X |
| 0111 | Valid | CW |
| 1000 | Valid | CW |
| 1001 | X | X |
| 1010 | X | X |
| 1011 | Valid | CCW |
| 1100 | X | X |
| 1101 | Valid | CCW |
| 1110 | Valid | CW |
| 1111 | X | X |